In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr

plt.style.use("default")
%matplotlib inline


In [ ]:
# Load synthetic telemetry
data = np.load("../data/raw/f1_telemetry.npy", allow_pickle=True).item()

telemetry = data["data"]
labels = data["labels"]
lap_index = data["lap_index"]

print("Telemetry shape:", telemetry.shape)
print("Labels shape:", labels.shape)
print("Positive samples:", labels.sum())


In [ ]:
sensor_names = [
    "Tire Temperature",
    "Tire Pressure",
    "Speed",
    "Throttle",
    "Brake",
    "RPM"
]

stats = {}

for i, name in enumerate(sensor_names):
    stats[name] = {
        "mean": telemetry[:,:,i].mean(),
        "std": telemetry[:,:,i].std(),
        "min": telemetry[:,:,i].min(),
        "max": telemetry[:,:,i].max()
    }

pd.DataFrame(stats).T


In [ ]:
sample_id = 0

plt.figure(figsize=(12,6))

for i, name in enumerate(sensor_names):
    plt.plot(telemetry[sample_id,:,i], label=name)

plt.title("Sample Telemetry (Single Lap Window)")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mean tire temperature per sample
mean_temp = telemetry[:,:,0].mean(axis=1)

plt.figure(figsize=(8,5))
sns.boxplot(x=labels, y=mean_temp)

plt.title("Mean Tire Temperature vs Wear Label")
plt.xlabel("Will Wear Within 12 Laps")
plt.ylabel("Mean Tire Temperature")
plt.show()


In [ ]:
corr, p = pearsonr(mean_temp, labels)

print(f"Correlation between mean tire temp and wear label: {corr:.4f}")
print(f"P-value: {p:.4e}")


In [ ]:
# Plot temperature trend for positive vs negative samples

pos_ids = np.where(labels == 1)[0][:5]
neg_ids = np.where(labels == 0)[0][:5]

plt.figure(figsize=(12,6))

for i in pos_ids:
    plt.plot(telemetry[i,:,0], color="red", alpha=0.6)

for i in neg_ids:
    plt.plot(telemetry[i,:,0], color="blue", alpha=0.6)

plt.title("Tire Temperature Trends\nRed = Wear Soon, Blue = No Wear")
plt.xlabel("Time Step")
plt.ylabel("Temperature")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.hist(lap_index, bins=30, edgecolor="black")

plt.title("Lap Index Distribution")
plt.xlabel("Lap Number")
plt.ylabel("Count")
plt.grid(True)
plt.show()


In [ ]:
# Approximate slope for tire temperature

seq_len = telemetry.shape[1]
idx = np.arange(seq_len)

slopes = []

for i in range(len(telemetry)):
    y = telemetry[i,:,0]
    slope = np.sum((idx - idx.mean())*(y - y.mean()))/np.sum((idx - idx.mean())**2)
    slopes.append(slope)

slopes = np.array(slopes)

plt.figure(figsize=(8,5))
sns.scatterplot(x=slopes, y=mean_temp, hue=labels, alpha=0.5)

plt.title("Temp Slope vs Mean Temp (Colored by Wear)")
plt.xlabel("Temperature Slope")
plt.ylabel("Mean Temperature")
plt.show()
